In [1]:
import numpy as np
import pandas as pd
import math


In [ ]:
train = np.load('./data/fashion_train.npy')

train_test = train[:, : -1]
y = train[:,-1]

print(train_test.shape, y.shape)

(10000, 784) (10000,)


In [ ]:



def input_layer(X_train, y_train):
    n_feartures = X_train.shape[1]
    output_nodes = len(set(y_train))
    len_of_X_train = len(X_train)
    return n_feartures, output_nodes, len_of_X_train


def hidden_layer(X, n_feartures, nunbers_of_nodes_hidden, len_of_X_train):
    #first hidden layer

    W = np.random.randn(n_feartures, nunbers_of_nodes_hidden)

    b = np.random.randn(len_of_X_train, nunbers_of_nodes_hidden)

    z = X@W+b

    a = np.maximum(0, z)

    return a


#output layer
def output_layer(a, output_nodes, len_of_X_train):

    W = np.random.randn(a.shape[1], output_nodes)

    b = np.random.randn(len_of_X_train, output_nodes)

    z = a@W+b

    softmax = np.exp(z - np.max(z))/np.sum(np.exp(z - np.max(z)))
    preds = np.argmax(softmax, axis=1)

    return print(f'pred: {np.argmax(softmax, axis=1)}, shape: {softmax.shape}'), preds


In [6]:
X = train_test

n_feartures, output_nodes, len_of_X_train = input_layer(X, y)

a = hidden_layer(X, n_feartures, nunbers_of_nodes_hidden=40, len_of_X_train=len_of_X_train)

preds = output_layer(a, output_nodes, len_of_X_train)

preds

pred: [0 0 0 ... 0 0 0], shape: (10000, 5)


(None, array([0, 0, 0, ..., 0, 0, 0], dtype=int64))

In [ ]:
def sigmoid(x):
    return np.exp(x) / (np.exp(x) + 1)

def derivative_sigmoid(x):
    sig = sigmoid(x)
    return sig * (1 - sig)

def softmax(z):
    return np.exp(z - np.max(z))/np.sum(np.exp(z - np.max(z)))

def softmax_derivative(z):
    s = np.exp(z) / np.sum(np.exp(z))
    s = s.reshape(-1, 1)
    jacobian = np.diagflat(s) - np.dot(s, s.T)
    return jacobian

def relu(x):
    return np.maximum(0, x)

def derivative_relu(x):
    return np.where(x > 0, 1, 0)

def cross_entropy(y, a, epsilon=1e-12):
    a = np.clip(a, epsilon, 1. - epsilon)
    return -np.sum(y * np.log(a))

def cross_entropy_derivative_a(y, a):
    return -1/a * np.sum(y)

def cross_entropy_derivative_softmax(y, a):
    return a-y


def binary_cross_entropy(y, a, epsilon=1e-12):
    a = np.clip(a, epsilon, 1. - epsilon)  # Avoid log(0) errors
    return -np.mean(y * np.log(a) + (1 - y) * np.log(1 - a))


In [35]:
derivative_dict = {
    softmax:softmax_derivative,
    cross_entropy:cross_entropy_derivative_a,
    sigmoid:derivative_sigmoid,
    relu:derivative_relu
}

In [ ]:
class Layer:
    def __init__(self,activation_function, previous_layer, size, X):
        self.activation_function = activation_function
        self.W = np.random.randn(X.shape[1], size)
        self.b = np.random.randn(X.shape[0], size)

        self.previous = previous_layer
        self.X = X

    def activate(self):
        self.Z = self.X.dot(self.W) + self.b
        self.a = self.activation_function(self.Z)
        return self.a

    def adjust_weights(self, alpha, loss_function,preds):
        P = derivative_dict[loss_function](preds,self.a) * derivative_dict[self.activation_function](self.Z) * self.previous.a
        self.W = self.W - alpha * P
        #self.W = self.W - alpha * derivative_dict[loss_function](self.next.a) * derivative_dict[self.next.activation_function](self.next.z)*self.a

# forward
l1 = Layer(relu,None,10,train_test)
X2 = l1.activate()
output_layer = Layer(sigmoid,l1,5,X2)
output_layer.activate()
preds = output_layer.a
#print(preds)
print(preds.shape)
print(output_layer.W.shape)

# backwards
output_layer.adjust_weights(0.01,cross_entropy,preds)

C:\Users\andra\AppData\Local\Temp\ipykernel_6776\3473699835.py:2: RuntimeWarning: overflow encountered in exp
  return np.exp(x) / (np.exp(x) + 1)
C:\Users\andra\AppData\Local\Temp\ipykernel_6776\3473699835.py:2: RuntimeWarning: invalid value encountered in divide
  return np.exp(x) / (np.exp(x) + 1)


(10000, 5)
(10, 5)


ValueError: operands could not be broadcast together with shapes (10000,5) (10000,10) 